In [31]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json
from APIWrapper import APIwrapper

In [32]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [33]:
jsondata={}

with open("healthcare_admissionRollingMean.json", "rt") as INFILE:
    jsondata["healthcare_admissionRollingMean"]=json.load(INFILE)
    
with open("healthcare_occupiedBedsRollingMean.json", "rt") as INFILE:
    jsondata["healthcare_occupiedBedsRollingMean"]=json.load(INFILE)
    
with open("cases_countRollingMean.json", "rt") as INFILE:
    jsondata["cases_countRollingMean"]=json.load(INFILE)
    
with open("autumn22_doses.json", "rt") as INFILE:
    jsondata["autumn22_doses"]=json.load(INFILE)
    
with open("autumn24_doses.json", "rt") as INFILE:
    jsondata["autumn24_doses"]=json.load(INFILE)


In [34]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")


def wrangle_data(rawdata):
    healthcare_admissionRollingMean = rawdata['healthcare_admissionRollingMean']  
    healthcare_occupiedBedsRollingMean = rawdata['healthcare_occupiedBedsRollingMean']
    cases_countRollingMean = rawdata ['cases_countRollingMean']
    autumn22_doses=rawdata['autumn22_doses']
    autumn24_doses=rawdata ['autumn24_doses']


    
    
    data={}
    for dataset in [healthcare_admissionRollingMean,healthcare_occupiedBedsRollingMean,cases_countRollingMean]:
        for entry in dataset:
            date=entry['date']
            metric=entry['metric']
            value=entry['metric_value']
            if date not in data:
                data[date]={}
            data[date][metric]=value
            
    dates=list(data.keys())
    dates.sort()
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['healthcare_admissionRollingMean','healthcare_occupiedBedsRollingMean','cases_countRollingMean'])
    
    metrics ={'healthcare_admissionRollingMean': 'COVID-19_healthcare_admissionRollingMean',
              'healthcare_occupiedBedsRollingMean': 'COVID-19_healthcare_occupiedBedsRollingMean',
              'cases_countRollingMean':'COVID-19_cases_countRollingMean'}
    
    for date, entry in data.items(): # each entry is a dictionary with cases, admissions and deaths
        pd_date=parse_date(date) # convert to Pandas format
        for column in ['healthcare_admissionRollingMean','healthcare_occupiedBedsRollingMean','cases_countRollingMean']: 
            metric_name=metrics[column]
            # do not assume all values are there for every date - if a value is not available, insert a 0.0
            value= entry.get(metric_name, 0.0)
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            df.loc[pd_date, column]=value
    df.fillna(0.0, inplace=True)

    data2={}
    for name in ['autumn22_doses','autumn24_doses']:
        dataset = rawdata[name]
        for entry in dataset:
             date=entry['date']
             age=entry['age']
             value=entry['metric_value']
             if date not in data2:
                 data2[date]={}
                 data2[date][age]=value
            
            
    dates=list(data2.keys())
    dates.sort()

    ages=[]
    for entry in data2.values():
        for k in entry.keys():
            if k not in ages:
                ages.append(k)
    ages.sort()
        


    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    autumn22_24=pd.DataFrame(index=index, columns=ages)
    
    metrics ={'autumn22_doses': 'COVID-19_vaccinations_autumn22_dosesByDay',
              'autumn24_doses': 'COVID-19_vaccinations_autumn24_dosesByDay'}
    
    for date, entry in data2.items(): 
        pd_date=parse_date(date) 
        for age in ages: 
            value= entry.get(age, 0.0)
            autumn22_24.loc[pd_date, age]=value
           
                
    
    autumn22_24.fillna(0.0, inplace=True)
    return df, autumn22_24







df, autumn22_24 = wrangle_data(jsondata)
#df = df.last('30D') # need to change because voila wasnt loading
#autumn22_24= autumn22_24.last('30D')

/tmp/ipykernel_765/1161078627.py:91: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  autumn22_24.fillna(0.0, inplace=True)


In [35]:
def poll(metricname):
    structure={"theme": "infectious_disease", 
           "sub_theme": "respiratory",
           "topic": "COVID-19",
           "geography_type": "Nation", 
           "geography": "England"}
    
    structure["metric"]= metricname
    
    api=APIwrapper(**structure)
    apidata=api.get_all_pages()
    
    print(f"Data points expected: {api.count}")
    print(f"Data points retrieved: {len(apidata)}")
    
    
    # with open(jsonfile,"wt") as OUTF:
    #     json.dump(apidata, OUTF)
        
    return apidata



def access_api():
    jsondata = {}
    
    jsondata['healthcare_admissionRollingMean']=poll('COVID-19_healthcare_admissionRollingMean')
    jsondata['healthcare_occupiedBedsRollingMean']= poll('COVID-19_healthcare_occupiedBedsRollingMean')
    jsondata['cases_countRollingMean']=poll('COVID-19_cases_countRollingMean')
    jsondata['autumn22_doses'] = poll('')
    jsondata['autumn24_doses'] = poll('')


    # jsondata={}
    # with open("healthcare_admissionRollingMean.json", "rt") as INFILE:
    #     jsondata["healthcare_admissionRollingMean"]=json.load(INFILE)
    
    # with open("healthcare_occupiedBedsRollingMean.json", "rt") as INFILE:
    #     jsondata["healthcare_occupiedBedsRollingMean"]=json.load(INFILE)
    
    # with open("cases_countRollingMean.json", "rt") as INFILE:
    #     jsondata["cases_countRollingMean"]=json.load(INFILE)
    
    # with open("autumn22_doses.json", "rt") as INFILE:
    #     jsondata["autumn22_doses"]=json.load(INFILE)
    
    # with open("autumn24_doses.json", "rt") as INFILE:
    #     jsondata["autumn24_doses"]=json.load(INFILE)
    # return jsondata
    


In [36]:
def api_button_callback(button):
    global df, autumn22_24
    apidata=access_api()
    df,autumn22_24=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    #apibutton.disabled=True
        
       

    
apibutton=wdg.Button(
    description='Press to refresh! :)', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    # FontAwesome names without the `fa-` prefix - try "download"
    icon=''
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets
display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Press to refresh! :)', style=ButtonStyle(), tooltip='')

In [37]:
def plot_ABC(scaletype):
    df[scaletype].plot()
    plt.show()
    
series=wdg.SelectMultiple(
    options=['healthcare_admissionRollingMean', 'healthcare_occupiedBedsRollingMean', 'cases_countRollingMean'],
    value=['healthcare_admissionRollingMean', 'healthcare_occupiedBedsRollingMean','cases_countRollingMean'],
    rows=3,
    description = 'Data',
    disabled=False
)

#---- widget 
scale_options = wdg.ToggleButtons(
    options=['linear','log'],
    description='scale:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],)


def ABC_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show()  
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


#----------------- refresh
def refresh_graph():
    one = scale_options.value
    if one == scale_options.options[0]:
        two = scale_options.options[1]
    else:
        two = scale_options.options[0]
    scale_options.value = two # for the redraw
    scale_options.value  = one # and change it back 
            
            
graph=wdg.interactive_output(ABC_graph, {'gcols': series, 'gscale': scale_options})
display(wdg.HBox([series, scale_options]), graph)

Output()

In [38]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_vaccination_data(rawdata):
    data2={}
    for name in ['autumn22_doses','autumn24_doses']:
        dataset = rawdata[name]
        for entry in dataset:
            date=entry['date']
            age=entry['age']
            value=entry['metric_value']
            if date not in data2:
                data2[date]={}
            data2[date][age]=value
            
            
    dates=list(data2.keys())
    dates.sort()

    ages=[]
    for entry in data2.values():
        for k in entry.keys():
            if k not in ages:
                ages.append(k)
    ages.sort()
        


    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    autumn22_24=pd.DataFrame(index=index, columns=ages)
    
    metrics ={'autumn22_doses': 'COVID-19_vaccinations_autumn22_dosesByDay',
              'autumn24_doses': 'COVID-19_vaccinations_autumn24_dosesByDay'}
    
    for date, entry in data2.items(): 
        pd_date=parse_date(date) 
        for age in ages: 
            value= entry.get(age, 0.0)
            autumn22_24.loc[pd_date, age]=value
           
                
    
    autumn22_24.fillna(0.0, inplace=True)
    return autumn22_24
    

#autumn22_24= autumn22_24.last('30D')

In [39]:
def poll2(metricname, jsonfile):
    structure = {"theme": "infectious_disease", 
           "sub_theme": "respiratory",
           "topic": "COVID-19",
           "geography_type": "Nation", 
           "geography": "England"}
    
    structure["metric"]= metricname
    
    
    api=APIwrapper(**structure)
    apidata=api.get_all_pages()
    
    print(f"Data points expected: {api.count}")
    print(f"Data points retrieved: {len(apidata)}")
    
    
    with open(jsonfile,"wt") as OUTF:
        json.dump(apidata, OUTF)
        
    return apidata



def access_api2():
    
    poll('COVID-19_vaccinations_autumn22_dosesByDay', 'autumn22_doses.json')
    poll('COVID-19_vaccinations_autumn24_dosesByDay', 'autumn24_doses.json')

    jsondata = {}
    with open("autumn22_doses.json", "rt") as INFILE:
        jsondata["autumn22_doses"] = json.load(INFILE)
    
    with open("autumn24_doses.json", "rt") as INFILE:
        jsondata["autumn24_doses"] = json.load(INFILE)
    
    return jsondata

def vaccination_callback(button):
    apidata = access_api2()
    global autumn22_24
    autumn22_24 = wrangle_vaccination_data(apidata)
    refresh_graph2()
    apibutton.icon="check"
    #apibutton.disabled=True

apibutton=wdg.Button(
    description='Press to refresh! :)', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    # FontAwesome names without the `fa-` prefix - try "download"
    icon=''
)

# remember to register your button callback function with the button
apibutton.on_click(vaccination_callback) # the name of your function inside these brackets
display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Press to refresh! :)', style=ButtonStyle(), tooltip='')

In [40]:
def plot_autumn22_24(scaletype):
    autumn22_24[scaletype].plot()
    plt.show()

year = wdg.SelectionSlider(
    options=autumn22_24.index.year.unique(),
    value=autumn22_24.index.year[-1],
    description='Years',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)


def refresh_graph2():
    current = year.value
    if current == year.options[0]:
        year.value = year.options[-1]
    else:
        year.value = year.options[0]
    year.value = current # for the redraw
    

def lineage_graph(graphyear):
    yeardf=autumn22_24[autumn22_24.index.year==graphyear]
    yearly= yeardf.groupby(pd.Grouper(freq='YE')).sum() 
    totals=yearly.sum(axis=1) 
    yearly=yearly.div(totals, axis=0)*100
    yearly = yearly[::-1]
    ax=yearly.plot(kind='bar', stacked=False,cmap='tab20')
    ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    ax.set_xticklabels(yearly.index.strftime('%b %y'),rotation= 0)
    ax.set_title('COVID-19 Vaccines by Age Group (2022-2024)')
    plt.show() 



Voutput=wdg.interactive_output(lineage_graph, {'graphyear': year})

display(year, Voutput)

SelectionSlider(continuous_update=False, description='Years', index=3, options=(2022, 2023, 2024, 2025), value…

Output()